In [1]:
import cv2
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from scipy import stats
from typing import List, Tuple
from tqdm import tqdm

import tensorflow
from tensorflow import keras
from tensorflow.keras import layers

Image = np.ndarray
CharacterWithLabel = tuple[Image, str]

In [2]:
import random

np.random.seed(1)
random.seed(1)

In [3]:
# Mounting to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
SOURCE_DIR = '/content/drive/MyDrive/cs4243-project/preprocessing/clean/all_relabelled'
BLACK_LINES_REMOVED_DIR = '/content/drive/MyDrive/cs4243-project/preprocessing/clean/all_relabelled/black_lines_removed'
BLACK_LINES_REMOVED_SUFFIX = '_cleaned'
CONTRAST_ENHANCED_DIR = '/content/drive/MyDrive/cs4243-project/preprocessing/clean/all_relabelled/contrast_enhanced'
THICKNESS_ENHANCED_DIR = '/content/drive/MyDrive/cs4243-project/preprocessing/clean/all_relabelled/thickness_enhanced'

## 1.0 Remove Black Lines

In [ ]:
def remove_black_lines(img_path, save=True, output_dir=BLACK_LINES_REMOVED_DIR, suffix=BLACK_LINES_REMOVED_SUFFIX) -> Image:
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Could not read {img_path}")
        return None

    lower_black = np.array([0, 0, 0])
    upper_black = np.array([8, 8, 8])
    mask = cv2.inRange(img, lower_black, upper_black)

    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=1)
    mask = cv2.erode(mask, kernel, iterations=1)
    cleaned = cv2.inpaint(img, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

    if save:
        fname = os.path.basename(img_path)
        name, ext = os.path.splitext(fname)
        save_path = os.path.join(output_dir, f"{name}{suffix}{ext}")
        cv2.imwrite(save_path, cleaned)
    return cleaned


def remove_black_lines_from_dir(input_dir: str = SOURCE_DIR) -> List[Image]:
  os.makedirs(BLACK_LINES_REMOVED_DIR, exist_ok=True)
  cleaned_images = []
  for image_path in glob.glob(os.path.join(input_dir, "*.png")):
    print(image_path)
    cleaned = remove_black_lines(image_path)
    print(cleaned.shape)
    cleaned_images.append(cleaned)
  return cleaned_images



## 2.0 Contrast Enhancement

In [ ]:
def enhance_contrast_ycrcb(img_path, output_dir, save=True):
    """Enhance image contrast using histogram equalization on Y (luminance) channel."""

    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Could not read {img_path}")
        return None

    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb)

    y_eq = cv2.equalizeHist(y)

    ycrcb_eq = cv2.merge((y_eq, cr, cb))
    enhanced = cv2.cvtColor(ycrcb_eq, cv2.COLOR_YCrCb2BGR)

    if save:
        # os.makedirs(output_dir, exist_ok=True)
        fname = os.path.basename(img_path)
        name, ext = os.path.splitext(fname)
        save_path = os.path.join(output_dir, f"{name}_enhanced{ext}")
        cv2.imwrite(save_path, enhanced)
        print(f"✅ Saved enhanced image to: {save_path}")

    return img, enhanced


def enhance_contrast_dir_input(input_dir = BLACK_LINES_REMOVED_DIR, output_dir = CONTRAST_ENHANCED_DIR, save=True):
  if save:
    os.makedirs(output_dir, exist_ok=True)

  for image_path in glob.glob(os.path.join(input_dir, "*.png")):
    enhance_contrast_ycrcb(image_path, output_dir, save=True)



## Segment into characters by colour clustering

In [ ]:
def merge_duplicates(char_info, iou_threshold=0.6):
    merged = []
    used = [False] * len(char_info)

    for i, info_i in enumerate(char_info):
        if used[i]:
            continue

        x1_i, y1_i, x2_i, y2_i = info_i['bbox']
        area_i = (x2_i - x1_i) * (y2_i - y1_i)
        merged_info = info_i

        for j, info_j in enumerate(char_info):
            if i == j or used[j]:
                continue
            x1_j, y1_j, x2_j, y2_j = info_j['bbox']
            area_j = (x2_j - x1_j) * (y2_j - y1_j)

            # intersection
            inter_x1, inter_y1 = max(x1_i, x1_j), max(y1_i, y1_j)
            inter_x2, inter_y2 = min(x2_i, x2_j), min(y2_i, y2_j)
            inter_w = max(0, inter_x2 - inter_x1)
            inter_h = max(0, inter_y2 - inter_y1)
            inter_area = inter_w * inter_h

            iou = inter_area / float(area_i + area_j - inter_area + 1e-5)
            if iou > iou_threshold:
                used[j] = True  # mark duplicate as used

        used[i] = True
        merged.append(merged_info)

    return merged

def split_overlap(mask):
    """Try vertical projection first, else fallback to watershed."""
    proj = np.sum(mask, axis=0)
    proj_smooth = cv2.GaussianBlur(proj, (7, 1), 0)

    valleys = np.where(proj_smooth < 0.3 * np.max(proj_smooth))[0]
    if len(valleys) > 0:
        split = valleys[len(valleys) // 2]
        return [mask[:, :split], mask[:, split:]]

    dist = cv2.distanceTransform(mask, cv2.DIST_L2, 5)
    _, fg = cv2.threshold(dist, 0.5 * dist.max(), 255, 0)
    fg = np.uint8(fg)
    unknown = cv2.subtract(mask, fg)
    _, markers = cv2.connectedComponents(fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    imgc = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(imgc, markers)
    pieces = []
    for i in np.unique(markers):
        if i <= 1:
            continue
        pieces.append(np.uint8(markers == i) * 255)
    return pieces

def segment_characters_color_overlap(image: Image, labels_string: str) -> List[CharacterWithLabel]:
    """
    Segment individual characters using color clustering + connected components,
    with overlap splitting and duplicate cleanup.
    Returns a list of (cropped_character_image, label) pairs.
    """
    if len(image.shape) != 3:
        raise ValueError("Expected a color image (BGR).")

    k = len(labels_string)

    # K-means clustering in HSV space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    pixels = hsv.reshape(-1, 3).astype(np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
    _, lbls, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    segmap = lbls.reshape(image.shape[:2])
    bg_color = np.median(hsv.reshape(-1, 3), axis=0)

    char_info = []
    all_widths, all_areas = [], []

    # cluster statistics
    for cid in np.unique(segmap):
        cluster_color = centers[cid]
        if np.linalg.norm(cluster_color - bg_color) < 25:
            continue

        mask = (segmap == cid).astype(np.uint8) * 255
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
        n, _, stats, _ = cv2.connectedComponentsWithStats(mask, 8)

        for i in range(1, n):
            x, y, w, h, area = stats[i]
            if area > 50 and w > 8 and h > 8:
                all_widths.append(w)
                all_areas.append(area)

    if len(all_widths) == 0:
        return []

    median_w = np.median(all_widths)
    median_area = np.median(all_areas)

    # bounding boxes
    for cid in np.unique(segmap):
        cluster_color = centers[cid]
        if np.linalg.norm(cluster_color - bg_color) < 25:
            continue

        mask = (segmap == cid).astype(np.uint8) * 255
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
        n, _, stats, _ = cv2.connectedComponentsWithStats(mask, 8)

        for i in range(1, n):
            x, y, w, h, area = stats[i]
            if area < 50 or w < 8 or h < 8:
                continue

            # overlapping blobs
            if w > 1.6 * median_w or area > 1.6 * median_area:
                submasks = split_overlap(mask[y:y+h, x:x+w])
            else:
                submasks = [mask[y:y+h, x:x+w]]

            for sm in submasks:
                coords = np.where(sm > 0)
                if len(coords[0]) == 0:
                    continue  # Skip empty submasks

                sm_y_min, sm_x_min = coords[0].min(), coords[1].min()
                sm_y_max, sm_x_max = coords[0].max() + 1, coords[1].max() + 1

                bbox_x1 = x + sm_x_min
                bbox_y1 = y + sm_y_min
                bbox_x2 = x + sm_x_max
                bbox_y2 = y + sm_y_max

                char_info.append({
                    "bbox": (bbox_x1, bbox_y1, bbox_x2, bbox_y2),
                    "mask": sm,
                    "cluster": int(cid)
                })

    # merge duplicates and sort left-to-right
    char_info = merge_duplicates(char_info)
    char_info.sort(key=lambda ci: ci["bbox"][0])

    if len(char_info) > len(labels_string):
        char_info = char_info[:len(labels_string)]
    elif len(char_info) < len(labels_string):
        labels_string = labels_string[:len(char_info)]

    labeled_characters: List[CharacterWithLabel] = []
    for info, label in zip(char_info, labels_string):
        x1, y1, x2, y2 = info["bbox"]
        cropped = image[y1:y2, x1:x2]
        labeled_characters.append((cropped, label))

    return labeled_characters


def segment_into_characters(image_dir: str = CONTRAST_ENHANCED_DIR) -> List[CharacterWithLabel]:
  segmented_character_with_labels: List[CharacterWithLabel] = []
  for filename in os.listdir(image_dir):
    print(filename)
    if not filename.endswith(".png"):
      continue
    print("passed")
    name, ext = os.path.splitext(filename)
    labels_string = name.split("-")[0]
    img = cv2.imread(os.path.join(image_dir, filename))
    if img is None:
      print(f"⚠️ Could not read {os.path.join(image_dir, filename)}")
      continue
    segmented = segment_characters_color_overlap(img, labels_string)
    segmented_character_with_labels.extend(segmented)
  return segmented_character_with_labels

## 4.1 Transform thin characters

In [ ]:
def transform_resolve_thin_characters(characters: List[CharacterWithLabel], is_save: bool = True):
  transformed = []
  output_dir = THICKNESS_ENHANCED_DIR
  if is_save:
      os.makedirs(output_dir, exist_ok=True) # Create output directory once

  for idx, (img, label) in enumerate(characters):
    if img is None:
      print(f"Warning: Skipping invalid image for label '{label}'")
      continue

    # --- Step 1: CLAHE on L channel (contrast enhancement) ---
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))
    enhanced = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

    # --- Step 2: Contrast stretching (deepens dark text without crushing highlights) ---
    min_val, max_val = np.min(enhanced), np.max(enhanced)
    stretched = cv2.convertScaleAbs(enhanced, alpha=255.0/(max_val - min_val), beta=-255.0*min_val/(max_val - min_val))

    # --- Step 3: Slight sharpening to improve text edge clarity ---
    blur = cv2.GaussianBlur(stretched, (0, 0), 1.2)
    sharpened = cv2.addWeighted(stretched, 1.4, blur, -0.4, 0)

    # --- Step 4: Subtle thickening (morphology only on dark regions) ---
    gray = cv2.cvtColor(sharpened, cv2.COLOR_BGR2GRAY)
    _, mask_dark = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)
    kernel = np.ones((2, 2), np.uint8)
    mask_dilated = cv2.dilate(mask_dark, kernel, iterations=1)
    mask_dilated = cv2.GaussianBlur(mask_dilated, (3, 3), 0)
    mask_dilated = mask_dilated.astype(float) / 255.0

    # Blend thickened regions into sharpened image
    thickened = sharpened.astype(float)
    for c in range(3):
        thickened[..., c] = thickened[..., c] * (1 - 0.15 * mask_dilated)  # darken dark strokes slightly

    thickened = np.clip(thickened, 0, 255).astype(np.uint8)
    transformed.append((thickened, label))

    # Save result if needed
    if is_save:
        save_path = os.path.join(output_dir, f"{idx:05d}_{label}.png")
        cv2.imwrite(save_path, thickened)
        # print(f"✅ Saved enhanced image to: {save_path}")

  return transformed



## 4.2 Deskew characters

In [ ]:

# ---------------------------------------------------------------
# PCA-based angle estimation (improved)
# ---------------------------------------------------------------
def estimate_angle_pca(mask: Image) -> float:
    """Estimate skew angle using PCA on character pixels."""
    mask_binary = (mask > 0).astype(np.uint8)
    # Invert if background is dark
    if np.mean(mask_binary) > 127:
        mask_binary = cv2.bitwise_not(mask_binary)

    coords = np.column_stack(np.where(mask_binary > 0))
    if len(coords) < 10:
        return 0.0

    mean, eigenvectors = cv2.PCACompute(coords.astype(np.float32), mean=None)
    if eigenvectors is None or len(eigenvectors) == 0:
        return 0.0

    v = eigenvectors[0]
    angle = np.degrees(np.arctan2(v[1], v[0]))

    # Normalize angle to [-45, 45] range
    if angle < -45:
        angle += 90
    elif angle > 45:
        angle -= 90

    return angle


# ---------------------------------------------------------------
# Projection-based angle estimation (alternative method)
# ---------------------------------------------------------------
def estimate_angle_projection(mask: Image) -> float:
    """Estimate angle using horizontal projection profile variance."""
    mask_binary = (mask > 0).astype(np.uint8)
    if np.mean(mask_binary) > 127:
        mask_binary = cv2.bitwise_not(mask_binary)

    h, w = mask_binary.shape
    if h < 10 or w < 10:
        return 0.0

    best_angle = 0.0
    max_variance = 0

    # Coarse search first (every 5 degrees)
    for angle in range(-30, 31, 5):
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(mask_binary, M, (w, h), borderValue=0)
        projection = np.sum(rotated, axis=1)
        variance = np.var(projection)

        if variance > max_variance:
            max_variance = variance
            best_angle = float(angle)

    # Fine search around best angle (every 1 degree)
    for angle in range(int(best_angle) - 4, int(best_angle) + 5, 1):
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(mask_binary, M, (w, h), borderValue=0)
        projection = np.sum(rotated, axis=1)
        variance = np.var(projection)

        if variance > max_variance:
            max_variance = variance
            best_angle = float(angle)

    return best_angle


# ---------------------------------------------------------------
# Improved deskew decision logic
# ---------------------------------------------------------------
def should_deskew(mask: Image, angle_threshold: float = 2.0, debug: bool = False) -> Tuple[bool, float]:
    """Determine if character should be deskewed."""
    angle_pca = estimate_angle_pca(mask)

    # If PCA gives unreliable result (0 or very small), try projection method
    if abs(angle_pca) < 1.0:
        angle_proj = estimate_angle_projection(mask)
        if abs(angle_proj) > 1.0:
            angle = angle_proj
        else:
            angle = angle_pca
    else:
        angle = angle_pca

    if debug:
        print(f"  Detected angle: {angle:.1f}°")

    # More lenient threshold - deskew if angle > threshold degrees
    if abs(angle) < angle_threshold:
        if debug:
            print(f"  Decision: NO DESKEW (angle too small)")
        return False, angle

    if debug:
        print(f"  Decision: DESKEW")
    return True, angle


# ---------------------------------------------------------------
# Apply rotation with proper border handling
# ---------------------------------------------------------------
def deskew_character(image: Image, mask: Image, debug: bool = False) -> Tuple[Image, float]:
    """Deskew a character image."""
    should_rotate, angle = should_deskew(mask, debug=debug)

    if not should_rotate or abs(angle) < 1.0:
        return image, 0.0

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # Calculate rotation matrix
    rot_matrix = cv2.getRotationMatrix2D(center, -angle, 1.0)

    # Calculate new dimensions to avoid clipping
    cos = np.abs(rot_matrix[0, 0])
    sin = np.abs(rot_matrix[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    # Adjust rotation matrix for new center
    rot_matrix[0, 2] += (new_w / 2) - center[0]
    rot_matrix[1, 2] += (new_h / 2) - center[1]

    # Set border color based on image type
    if len(image.shape) == 3:
        border_color = (255, 255, 255)  # White for BGR
    else:
        border_color = 255  # White for grayscale

    # Rotate with white background
    rotated = cv2.warpAffine(image, rot_matrix, (new_w, new_h),
                            flags=cv2.INTER_CUBIC, borderValue=border_color)

    # Crop to remove white borders by finding bounding box of content
    if len(rotated.shape) == 3:
        gray = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
    else:
        gray = rotated

    # Find non-white regions
    _, binary = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY_INV)
    coords = np.column_stack(np.where(binary > 0))

    if len(coords) > 0:
        y_min, x_min = coords.min(axis=0)
        y_max, x_max = coords.max(axis=0)

        # Add small padding
        padding = 2
        y_min = max(0, y_min - padding)
        x_min = max(0, x_min - padding)
        y_max = min(rotated.shape[0], y_max + padding + 1)
        x_max = min(rotated.shape[1], x_max + padding + 1)

        rotated = rotated[y_min:y_max, x_min:x_max]

    return rotated, angle



def deskew_characters(characters_with_labels: List[CharacterWithLabel], debug: bool = False, visualize: bool = False) -> List[CharacterWithLabel]:
    deskewed: List[CharacterWithLabel] = []

    for idx, (char_img, label) in enumerate(characters_with_labels):
        gray = cv2.cvtColor(char_img, cv2.COLOR_BGR2GRAY) if len(char_img.shape) == 3 else char_img
        _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        if debug:
            print(f"\nCharacter {idx + 1} ('{label}'):")

        rotated, angle = deskew_character(char_img, mask, debug=debug)
        deskewed.append((rotated, label))

        if visualize:
            fig, axes = plt.subplots(1, 2, figsize=(6, 3))
            axes[0].imshow(cv2.cvtColor(char_img, cv2.COLOR_BGR2RGB))
            axes[0].set_title(f"Before '{label}'")
            axes[0].axis("off")
            axes[1].imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))
            axes[1].set_title(f"After ({angle:.1f}°)")
            axes[1].axis("off")
            plt.tight_layout()
            plt.show()

    return deskewed

# 4.3 Smoothen edges, random rotate, random zoom, add gaussian noise

In [ ]:
SMOOTHENED_DIR = "/content/drive/MyDrive/cs4243-project/preprocessing/clean/all_relabelled/outputs/smoothed_characters"
ROTATED_DIR = "/content/drive/MyDrive/cs4243-project/preprocessing/clean/all_relabelled/outputs/rotated_characters"
ZOOMED_DIR = "/content/drive/MyDrive/cs4243-project/preprocessing/clean/all_relabelled/outputs/zoomed_characters"
NOISED_DIR = "/content/drive/MyDrive/cs4243-project/preprocessing/clean/all_relabelled/outputs/noised_characters"

def transform_smoothen_edges(characters: List[CharacterWithLabel], is_save: bool = True):
    transformed = []
    output_dir = SMOOTHENED_DIR
    if is_save:
        os.makedirs(output_dir, exist_ok=True)

    for idx, (img, label) in enumerate(characters):
        if img is None:
            print(f"⚠️ Warning: Skipping invalid image for label '{label}'")
            continue

        # --- Step 1: Edge-preserving smoothing ---
        # This keeps color edges sharp while smoothing inside regions
        smoothed = cv2.edgePreservingFilter(img, flags=1, sigma_s=40, sigma_r=0.3)

        # --- Step 2: Optional mild bilateral smoothing to further refine edges ---
        smoothed = cv2.bilateralFilter(smoothed, d=7, sigmaColor=50, sigmaSpace=50)

        # --- Step 3: Optional gentle Gaussian blur to remove remaining roughness ---
        smoothed = cv2.GaussianBlur(smoothed, (3, 3), 0.5)

        transformed.append((smoothed, label))

        # --- Step 4: Save result if needed ---
        if is_save:
            save_path = os.path.join(output_dir, f"{idx:05d}_{label}.png")
            cv2.imwrite(save_path, smoothed)

    return transformed

def transform_random_rotate(characters: List[CharacterWithLabel], is_save: bool = True):
    transformed = []
    output_dir = ROTATED_DIR
    if is_save:
        os.makedirs(output_dir, exist_ok=True)

    for idx, (img, label) in enumerate(characters):
        if img is None:
            print(f"Warning: Skipping invalid image for label '{label}'")
            continue

        h, w = img.shape[:2]

        # --- Step 1: Randomly choose a rotation angle between -20° and +20° ---
        angle = random.uniform(-20, 20)

        # --- Step 2: Compute rotation matrix around the image center ---
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)

        # --- Step 3: Apply rotation using affine transform ---
        # Use border mode to fill outside regions smoothly (instead of black)
        rotated = cv2.warpAffine(
            img,
            M,
            (w, h),
            flags=cv2.INTER_LINEAR,
            borderMode=cv2.BORDER_REPLICATE
        )

        transformed.append((rotated, label))

        # --- Step 4: Save result if needed ---
        if is_save:
            save_path = os.path.join(output_dir, f"{idx:05d}_{label}.png")
            cv2.imwrite(save_path, rotated)

    return transformed

MAX_ZOOM_OUT_FACTOR = 0.8
MAX_ZOOM_IN_FACTOR = 1.2
def transform_random_zoom(characters: List[CharacterWithLabel], is_save: bool = True):
    transformed = []
    output_dir = ZOOMED_DIR
    if is_save:
        os.makedirs(output_dir, exist_ok=True)

    for idx, (img, label) in enumerate(characters):
        if img is None:
            print(f"Warning: Skipping invalid image for label '{label}'")
            continue

        h, w = img.shape[:2]

        # --- Step 1: Choose random zoom factor (e.g., between 0.9x and 1.1x) ---
        zoom_factor = random.uniform(MAX_ZOOM_OUT_FACTOR, MAX_ZOOM_IN_FACTOR)

        # --- Step 2: Compute new size after zoom ---
        new_w, new_h = int(w * zoom_factor), int(h * zoom_factor)
        resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

        # --- Step 3: Crop or pad to restore original size ---
        if zoom_factor < 1.0:
            # Zoomed out → need to pad
            pad_w = (w - new_w) // 2
            pad_h = (h - new_h) // 2
            zoomed = cv2.copyMakeBorder(
                resized,
                pad_h,
                h - new_h - pad_h,
                pad_w,
                w - new_w - pad_w,
                borderType=cv2.BORDER_REPLICATE
            )
        else:
            # Zoomed in → need to crop
            start_x = (new_w - w) // 2
            start_y = (new_h - h) // 2
            zoomed = resized[start_y:start_y + h, start_x:start_x + w]

        transformed.append((zoomed, label))

        # --- Step 4: Save result if needed ---
        if is_save:
            save_path = os.path.join(output_dir, f"{idx:05d}_{label}.png")
            cv2.imwrite(save_path, zoomed)

    return transformed

def transform_add_gaussian_noise(characters: List[CharacterWithLabel], is_save: bool = True):
    transformed = []
    output_dir = NOISED_DIR
    if is_save:
        os.makedirs(output_dir, exist_ok=True)

    for idx, (img, label) in enumerate(characters):
        if img is None:
            print(f"Warning: Skipping invalid image for label '{label}'")
            continue

        # --- Step 1: Normalize image to float32 for noise addition ---
        img_float = img.astype(np.float32) / 255.0

        # --- Step 2: Generate Gaussian noise ---
        # mean = 0, stddev between 0.01 and 0.03 (light noise)
        noise_std = random.uniform(0.01, 0.03)
        noise = np.random.normal(0, noise_std, img_float.shape).astype(np.float32)

        # --- Step 3: Add noise and clip back to valid range ---
        noisy = np.clip(img_float + noise, 0.0, 1.0)

        # --- Step 4: Convert back to uint8 ---
        noisy_uint8 = (noisy * 255).astype(np.uint8)

        transformed.append((noisy_uint8, label))

        # --- Step 5: Save result if needed ---
        if is_save:
            save_path = os.path.join(output_dir, f"{idx:05d}_{label}.png")
            cv2.imwrite(save_path, noisy_uint8)

    return transformed
